In [ ]:
import numpy as np
import pandas as pd
import csv

In [ ]:
dataset = pd.read_csv('Train_E.csv')
#handling missing values
dataset = dataset.fillna(dataset.mean())

#one hot encoding for categorical columns (iso_code, continent, location)

dummy_iso_code = pd.get_dummies(dataset['iso_code'], prefix = "iso_code")
dummy_continent = pd.get_dummies(dataset['continent'], prefix = "continent")
dummy_location = pd.get_dummies(dataset['location'], prefix = "location")
#print(dummy_iso_code.head())
#print(dummy_continent.head())
#print(dummy_location.head())
dataset = pd.concat([dummy_location, dataset], axis = 1)
dataset = pd.concat([dummy_continent, dataset], axis = 1)
dataset = pd.concat([dummy_iso_code, dataset], axis = 1)
dataset = dataset.drop(['iso_code', 'continent', 'location'], axis = 1)
dataset.head()

,iso_code_ABW,iso_code_AFG,iso_code_AGO,iso_code_ALB,iso_code_ARE,continent_Africa,continent_Asia,continent_Europe,continent_North America,location_Afghanistan,location_Albania,location_Angola,location_Aruba,location_United Arab Emirates,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,stringency_index,population,population_density,median_age,aged_65_older,aged_70_older,gdp_per_capita,diabetes_prevalence,life_expectancy
0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,13-03-2020,2.000000,2.000000,138.06281,0.00000,0.000000,2.162243,18.733000,18.733000,46.428623,0.000000,0.000000,0.425036,0.00,106766,584.8,41.2,13.085,7.452,35973.781,11.62,76.29
1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,19-03-2020,12729.909957,141.193074,0.28600,203.09697,2.180952,0.000000,2444.184356,47.163528,2.676000,25.534926,0.438827,0.000000,33.33,106766,584.8,41.2,13.085,7.452,35973.781,11.62,76.29
2,1,0,0,0,0,0,0,0,1,0,0,0,1,0,20-03-2020,4.000000,2.000000,0.28600,0.00000,0.000000,0.000000,37.465000,18.733000,2.676000,0.000000,0.000000,0.000000,33.33,106766,584.8,41.2,13.085,7.452,35973.781,11.62,76.29
3,1,0,0,0,0,0,0,0,1,0,0,0,1,0,21-03-2020,12729.909957,141.193074,0.28600,203.09697,2.180952,0.000000,2444.184356,47.163528,2.676000,25.534926,0.438827,0.000000,44.44,106766,584.8,41.2,13.085,7.452,35973.781,11.62,76.29
4,1,0,0,0,0,0,0,0,1,0,0,0,1,0,22-03-2020,12729.909957,141.193074,0.28600,203.09697,2.180952,0.000000,2444.184356,47.163528,2.676000,25.534926,0.438827,0.000000,44.44,106766,584.8,41.2,13.085,7.452,35973.781,11.62,76.29
